<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/133_%E3%83%87%E3%83%90%E3%83%83%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

デバッグ
========

Python デバッガ
---------------

**バグ**（bug）すなわちプログラムの誤りや欠陥を特定して取り除き、動作を仕様通りのものとするための作業を**デバッグ**（debug）という。デバッグを支援するツールを**デバッガ**（debugger）と呼ぶ。

標準ライブラリの `pdb` モジュールは、Python プログラム用の対話型ソースコードデバッガを提供する。ソースコードの適当な箇所（複数も可）に次の文を挿入する:

``` python
import pdb; pdb.set_trace()
```

スクリプトの実行時に、`pdb.set_trace()` 関数は Python デバッガに入る。

`pdb.set_trace()` 関数を直接呼び出すのではなく、組み込み関数 `breakpoint()` を使用してもよい。

``` python
breakpoint(*args, **kws)
```

この組み込み関数は、`sys.breakpointhook()` 関数を `args` と `kws` をそのまま渡して呼び出す。

`sys.breakpointhook()` 関数のデフォルトの実装は次のとおり。

  * 最初に環境変数 `PYTHONBREAKPOINT` を調べる。
  * この環境変数が `'0'` に設定されていた場合は、この関数は直ちに終了し、何もしない。
  * この環境変数が設定されていないか、空文字列に設定されていた場合は、 `pdb.set_trace()` 関数が呼ばれる。
  * それ以外の場合は、`PYTHONBREAKPOINT` に設定された名前の関数を `args` と `kws` をそのまま渡して呼び出す。その戻り値は `breakpoint()` 関数に返される。

たとえば、`sys.breakpointhook()` をサードパーティ製の Python デバッガを起動する関数に置き換えると、`breakpoint()` の実行でサードパーティ製の Python デバッガに入ることができる。次のコードは、 [ipdb](https://pypi.org/project/ipdb/) を使用するコード例である。

``` python
import ipdb

# 環境変数 PYTHONBREAKPOINT に "ipdb.set_trace" を設定しても同じことができる
sys.breakpointhook = ipdb.set_trace()

def foo():
    breakpoint()
    ...

def main():
    foo()

if __name__ == "__main__":
    main()
```

以降では、`pdb.set_trace()` や `breakpoint()` の挿入個所を**ブレークポイント**と呼ぶ。

また、コマンドラインで

``` text
PS > python -m pdb myscript.py
```

のように実行すると、`myscript.py` が異常終了した際に自動的に Python デバッガに入る。

Python デバッガでの主なコマンドは次のとおり（コマンドの括弧は省略できることを意味する）。

| コマンド | 機能 |
|:---|:---|
| `p 変数` | 変数の値を出力する |
| `c(ont(inue))` | 次のブレークポイントまで実行する。次のブレークポイントがなければ最後まで実行して終了する |
| `n(ext)` | 1 行実行して次の行に進む。ただし関数の中には入らない |
| `s(tep)` | 1 行実行して次の行に進む。関数の中にも入る |
| `r(eturn)` | 関数の終わりまで実行する |
| `run` | デバッグを再起動する |
| `q(uit)` | デバッガーを終了する |
| `h(elp) [コマンド]` | コマンドの一覧を表示する。引数としてコマンドが与えられた場合、そのコマンドのヘルプが表示される |

たとえば、次のスクリプトを実行すると、`breakpoint()` 関数の呼び出し箇所で Python デバッガに入る：

``` python
def double(x):
   breakpoint()
   return x * 2
val = 3
print(f"{val} * 2 is {double(val)}")
```

Python デバッガに入ると、`->` で実行中の行が示され、`(Pdb) ` でコマンド入力待ちが示される。`p x` コマンドを入力すると変数 `x` の値が表示され、`c` コマンドを入力するとスクリプトが最後まで実行される。

``` python
> ...(3)double()
-> return x * 2
(Pdb) p x
3
(Pdb) c
3 * 2 is 6
```

プログラムが異常終了して Python デバッガに入った場合、 `c` コマンドを入力するとエラー内容が表示される。

デバッグ作業が終了したら、ソースコードに挿入したブレークポイントを削除する。削除し忘れると、その部分でコードの実行が停止してしまう。

VSCode の Python 拡張機能に付属するデバッガーを使うなら、スクリプトに直接 `breakpoint()` を書くことなく、ブレークポイントの設置とデバッグモードへの移行が可能である。

エディタ画面の左端をクリックすると、その右側の行にブレークポイントが設定される。エディタ画面右上の ▷ のメニューからデバッガーを起動できる。デバッガーを操作する 6 つのアイコンが表示され、左から、続行（`c` コマンドと同じ）、ステップオーバー（`n` コマンドと同じ）、ステップイン（`s` コマンドと同じ）、ステップアウト（`r` コマンドと同じ）、再起動（`run` コマンドと同じ）、停止（`q` コマンドと同じ）となる。操作は、あまつ楓さんの YouTube 動画で見ることができる。

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo('LdwMM26B0zI?start=15', width=640, height=360)

ログ出力
--------

プログラムがエラーになった場合、エラーになった処理を特定し、エラーを再現してバグを発見する。このような作業を行うには、プログラムの処理過程を一定の形式で時系列に記録されたデータが欠かせない。このような記録を**ログ**（log）という。ログを残すことを**ロギング**（logging）とか、ログを取るとかいう。

ログは、エラーを再現するのに十分な情報を残すだけでなく、検索しやすいようにメタ情報（日時や重要度など）を付けることが望ましい。これらを全て `print()` 関数だけで実現するのは容易ではない。また、ログを永続的に保存したい場合、ファイル管理も必要となる。

そこで、ロギングの実装を簡単にするフレームワークを利用することになる。Python では標準のロギングフレームワークとして `logging` モジュールが標準ライブラリに含まれている。

### ログレベル ###

`logging` では、ログの発生事由の重大度を表す**ログレベル**と呼ばれる定数が以下のように設定されている。

| ログレベル | 値 | 意味 |
|:---|:--:|:---|
| `logging.CRITICAL` | 50 | プログラム自体が実行を続けられないことを表す致命的エラーの情報 |
| `logging.ERROR` | 40 | 重大な問題により、プログラムのある機能が実行されないことの記録 |
| `logging.WARNING` | 30 | 想定外のことが起こった、または問題が近く起こりそうなことの情報（エラーになる前の警告） |
| `logging.INFO` | 20 | 正常動作のプロセスを示す記録 |
| `logging.DEBUG` | 10 | おもに問題を診断するときにのみ関心があるような、詳細な情報（開発時のデバッグだけで使用するもの） |
| `logging.NOTSET` | 0 | 変数値や詳細情報などすべての情報 |

### ロガー ###

`logging` では、 `logging.Logger` オブジェクト（以下、ロガーと略す）がログを記録するときのインターフェースになっている。

ロガーは、その `name` 属性（値は空でない文字列）によって同じプロセス上で一意に存在する。つまり、**ある名前を持つロガーはシングルトンである**。 `logging` モジュールが読み込まれた時点で、 `name` 属性が `"root"` となっているロガーが作成され、**ルートロガー**と呼ばれる。

`logging.Logger` クラスを直接インスタンス化することは絶対にしてはならず、コードからは常に次のモジュール関数経由でロガーにアクセスしなければならない。

``` python
logging.getLogger(name=None)
```

この関数は、 `name` 引数で指定された名前を持つロガーが存在しなければ新規に作成して返し、ロガーが既存であるならばそれを返す。`name` の指定を省略するか、 `None` を指定した場合は、ルートロガーを返す。ロガーはシングルトンであるから、与えられた `name` に対して、この関数はどの呼び出しでも同じロガーを返す。したがって、ロガーをユーザー定義関数の引数に渡したり、ユーザー定義クラスのインスタンス属性として保持することは意味がないことに注意する。

ロガーの名前は、その文字列にドット `.` を含めることで、階層を表す。たとえば、 `name` を `'a.b.c'` と指定すると、次のようなロガー階層を指定したことになる。

``` text
  ┌────┐
  │ ルート │
  └────┘
       │
  ┌────┐
  │   a    │
  └────┘
親     │
↑┌────┐
  │   b    │
↓└────┘
子     │
  ┌────┐
  │   c    │←この階層を指定
  └────┘
```

ロガー階層は木構造をなし、ルートロガーがその最上位（ルート）にある。上位のロガーを**親ロガー**と呼び、下位のロガーを**子ロガー**と呼ぶ。親ロガーより上位のロガーは**先祖ロガー**、子ロガーより下位のロガーは**子孫ロガー**と呼ぶ。

ロガー階層を使用することで、必要に応じて下流のコードできめ細かい制御が可能になる。ロガー階層を決める簡潔な方法は、 Python パッケージ階層と一致させることである。これは次のように書けばよい:

``` python
import logging
logger = logging.getLogger(__name__)
```

このような名前を持つロガーを**モジュールレベルロガー**と呼ぶ。モジュールレベルロガーなら、名前だけで、どこでイベントのログが取られたか、直感的に明らかになる。各モジュールでは、このモジュールレベルロガーを使うべきである。

モジュールより粒度が小さいロガーを大量に作成すると、シングルトンであるロガーはスクリプトの実行中に存続するため、メモリが解放されることなく消費されることに注意する。

### フォーマッター ###

ロガーは、ログ記録するための情報を `logging.LogRecord` クラスのインスタンスに変換する。 `logging.LogRecord` のインスタンス属性の中から必要な情報を選んで書式化する作業は、 `logging.Formatter` クラスで実装されている。コードからは `logging.Formatter` クラスのインスタンス（以下、フォーマッターと略す）に対し `logging.LogRecord` のインスタンス属性に対応する %-形式書式を含んだ書式文字列を与えることで、ログメッセージの書式を制御する。 `logging.LogRecord` のインスタンス属性と %-形式書式の対応関係は次のとおり。

| 属性 | 意味 | %-形式書式 |
|:---|:---|:---|
| `name` | ロガーの名前（文字列） | `%(name)s` |
| `asctime` | `logging.LogRecord` インスタンスが生成された時刻で `'2023-07-08 16:49:45,896'` 形式の文字列 | `%(asctime)s` |
| `created` | `logging.LogRecord` インスタンスが生成された時刻で `time.time()` によって返される形式の浮動小数点数 | `%(created)f` |
| `levelname` | ログレベルを指す文字列（`'DEBUG'`, `'INFO'`, `'WARNING'`, `'ERROR'`, `'CRITICAL'`） | `%(levelname)s` |
| `levelno` | ログレベルを指す数値を表す文字列 | `%(levelno)s` |
| `message` | ログメッセージ（文字列） | `%(message)s` |
| `pathname` | ログ発生ファイルの完全なパスを表す文字列（利用できる場合のみ） | `%(pathname)s` |
| `filename` | ログ発生ファイルの名前（`pathname` のファイル名部分） | `%(filename)s` |
| `module` | ログ発生モジュールの名前（`filename` の名前部分） | `%(module)s` |
| `funcName` | ログ発生関数の名前（文字列） | `%(funcName)s` |
| `lineno` | ログ発生行番号である整数（利用できる場合のみ） | `%(lineno)d` |
| `process` | プロセス ID （利用可能な場合のみ） | `%(process)d` |
| `processName` | プロセス名 （利用可能な場合のみ） | `%(processName)s` |
| `thread` | スレッド ID （利用可能な場合のみ） | `%(thread)d` |
| `threadName` | スレッド名（利用可能な場合のみ） | `%(threadName)s` |
| `taskName` | `asyncio.Task` 名 (利用可能な場合のみ)。Python 3.12 で追加 | `%(taskName)s` |
| `msecs` | `logging.LogRecord` インスタンスが生成された時刻のミリ秒部分（整数） | `%(msecs)d` |
| `relativeCreated` | `logging` モジュールが読み込まれた時刻からの経過ミリ秒（整数） | `%(relativeCreated)d` |

`logging.Formatter` クラスのインスタンス化は次のとおり。。

``` python
logging.Formatter(fmt=None, datefmt=None, style='%', validate=True, *, defaults=None)
```

| 引数 | 意味 |
|:---|:---|
| `fmt` | メッセージに対する書式文字列。`fmt` が指定されない場合、`'%(message)s'` が使われる |
| `datefmt` | メッセージの日付/時刻部分のための書式文字列。`time.strftime()` が受け付けるものを使う。`datefmt` が指定されない場合、<br />`'%Y-%m-%d %H:%M:%S,uuu'` が使われる |
| `style` | プレースホルダーの形式。次のいずれかである<br /><br />・`'%'`: `'%(attrname)'` 形式（デフォルト）<br /><br />・`'{'`: `{attrname}` 形式: `str.format()` と互換となる<br /><br />・`'$'`: `${attrname}` 形式: テンプレート文字列と互換となる |
| `validate` | `True`（デフォルト）の場合、`style` と `fmt` が不正だったりミスマッチだった場合に `ValueError` 例外を送出する |
| `defaults` | カスタムフィールドとして使用されるデフォルトの値を辞書形式で指定する |

% 形式の文字列書式化 `'%(attrname)'` 形式がデフォルトになっている。`style='{'` を指定することにより、`format()` ないし f-string で見慣れた書式を使うことはできる。

次のメッセージ書式文字列は、人が読みやすい形式の時刻、メッセージの深刻度、およびメッセージの内容を、順番に出力することを指定する:

``` python
'%(asctime)s - %(levelname)s - %(message)s'
```

### フィルター ###

**フィルター**は、どのログ記録を出力するかを決定する。フィルターには次の 2 種類がある。

  1. `logging.Filter` オブジェクト:  
ロガーの名前 `name` を使って `logging.Filter(name)` でインスタンス化する。ログ記録の主体が、`name` と一致する名前のロガーであるか、またはその子孫ロガーである場合にのみ、ログが出力されるようになる。たとえば、 `name` に `'A.B'` を指定してインスタンス化されたフィルターは、ロガー `'A.B'`, `'A.B.C'`, `'A.B.C.D'`, `'A.B.D'` 等によって記録されたイベントは許可するが、`'A.BB'`, `'B.A.B'` などは許可しない。 `name` に空の文字列を指定してインスタンス化された場合、すべてのイベントを通過させる。
  2. 呼び出し可能オブジェクト:  
フィルターとする呼び出し可能オブジェクトは 1 つの引数を取る必要があり、その引数に `logging.LogRecord` のインスタンスが渡される。`logging.LogRecord` インスタンスの `getMessage()` メソッドを呼び出すとメッセージが返されるので、そのメッセージがログとして出力すべきときは呼び出し可能オブジェクトが `True` を返すようにし、出力すべきではないときは `False` を返すようにする。

### ハンドラ ###

ロガーは、ログ記録の出力処理を `logging.Handler` クラスのサブクラスのインスタンス（以下、ハンドラと略す）に委ねる。ハンドラは、デフォルトでは全てのログ記録を `'%(message)s'` で書式化して出力する。以下のメソッドで出力を設定することができる。

| メソッド | 機能 |
|:---|:---|
| `setLevel(level)` | このハンドラで出力される最低のログレベルを `level` に設定する。デフォルトでは最低のログレベルが `NOTSET` に設定される |
| `setFormatter(fmt)` | このハンドラのフォーマッターを `fmt` に設定する。デフォルトでは `Formatter()` が使われる |
| `addFilter(filter)` | 指定されたフィルター `filter` をこのハンドラに追加する。このメソッドを何回も呼び出して複数の異なるフィルターを追加することも<br />できる。ハンドラは、全てのフィルターを通過したログ記録のみ出力する |
| `removeFilter(filter)` | 指定されたフィルター `filter` をこのハンドラから除去する |

`logging.Handler` クラスは抽象基底クラスなので、直接インスタンス化して使用するものではない。 `logging` モジュールは、以下の 3 つの `logging.Handler` クラスのサブクラスを提供している。

  * `logging.StreamHandler`
  * `logging.FileHandler`
  * `logging.NullHandler`

``` python
logging.StreamHandler(stream=None)
```

ログ出力先をファイルを使用しないストリーム（ `sys.stdout`, `sys.stderr` など）とするハンドラを返す。

| 引数 | 意味 |
|:---|:---|
| `stream` | `write()` および `flush()` メソッドをサポートするオブジェクト（つまり書き込み可能なストリーム）を指定しなければならない。`stream` を指定しなけれ<br />ば、`sys.stderr` が使われる |

以下のコードは、ハンドラへのフォーマット設定とフィルター設定の例である。

In [ ]:
import logging
import sys

fmt = "%(asctime)s - %(name)s - %(levelname)s - %(message)s"

def nopasswordfilter(record: logging.LogRecord) -> bool:
    log_message = record.getMessage()
    return "password" not in log_message

st_handler = logging.StreamHandler(sys.stdout)  # Colab で出力するため出力先を標準出力とする
st_handler.setLevel(logging.DEBUG)
st_handler.setFormatter(logging.Formatter(fmt))  # 書式を fmt に指定
st_handler.addFilter(nopasswordfilter)  # "password" を含むメッセージをログ出力しない

``` python
logging.FileHandler(file, mode='a', encoding=None, delay=False, errors=None)
```

ログ出力先をファイルシステム上のファイルとするハンドラを返す。

| 引数 | 意味 |
|:---|:---|
| `file` | `open()` 関数における同名の引数と同じ役割 |
| `mode` | `open()` 関数における同名の引数と同じ役割。デフォルトは `'a'`（追記） |
| `encoding` | `open()` 関数における同名の引数と同じ役割（日本語を出力する場合、`encoding='utf-8'` を指定しないとエラーとなることも同様） |
| `delay` | `True` の場合、ファイルのオープンは `emit()` メソッドの最初の呼び出しまで遅延される |
| `errors` | `open()` 関数における同名の引数と同じ役割 |

``` python
logging.NullHandler()
```

いかなるログ出力も行わないというハンドラを返す。

ロガーに対して、いかなるハンドラの設定も与えない場合、ロガーはモジュール属性 `logging.lastResort` に設定されたハンドラに接続して出力処理を委ねる。デフォルトでは、 `logging.lastResort` はレベルが `WARNING` にセットされた、メッセージを単に `sys.stderr` に出力するハンドラである。

`logging` を使うライブラリを開発する場合、ハンドラーやフォーマッター、フィルターを追加してログ出力をカスタマイズするのはライブラリ開発者ではなく、アプリケーション開発者の責務であることに注意する。ライブラリを使用するアプリケーションが `logging` をまったく使わないということも許容されるのであって、この場合、ライブラリコードがログ記録を作り出すロガーのメソッドを呼び出すと、 `logging.lastResort` が発動する。すなわち、重大度 `WARNING` 以上のイベントが、`sys.stderr` に表示される。

この表示を回避するには、`NullHandler` のインスタンスをライブラリの名前空間で使われるトップレベルロガーに取り付ける。 `NullHandler` 以外のハンドラを追加しないことが強く推奨されている。

### 有効レベルとイベントの伝播 ###


ロガーは、出力処理を委ねるハンドラを取り付けるメソッドを持つ。また、最低のログレベルとフィルターはロガーでも設定できる。次の図は、ロガーにもフィルターを設定した場合でのロガーとハンドラにおけるおおざっぱなワークフローを示す。

![](http://www.plantuml.com/plantuml/png/SoWkIImgAStDuR9wtBpmSNFpuwRTZvltF6xQOSsLcQQWWIRuk7dDu-QPZvjNF-fS_hXvxUCc87SFND1UKw4a8pLFGQCojLYJIq71oYS_FIWrERyenPehDQSuLK4ZBnyaNboINy2LcfUIcGQn0s7GrCTDYu46FMxQ3-IY68VBW0AWypDBClFp59GUD_S_RjxykBdpSVDA9OLgmkv7ACVf1OWP28LudQW015PW5Il1lfa7j1gYpXeeX-m1Y1U03B0dSFcjbiiA73ORCAB60QW066a0)

ロガーに複数のハンドラを取り付けることができる。また、複数のロガーを使用する場合に、それらに同じハンドラを追加して、出力処理を共有することもできる。

ログレベル、ハンドラに関するロガーの属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `level` | このロガーのレベルしきい値。この属性を直接設定してはならない |
| `handlers` | このロガーに直接取り付けられたハンドラーのリスト。この属性を直接設定してはならない |
| `propagate` | イベントの伝播を制御するための真偽値 |

ログレベル、ハンドラ、フィルターに関するロガーのメソッドは次のとおり。

| メソッド | 機能 |
|:---|:---|
| `setLevel(level)` | ハンドラに委ねられる最低のログレベルを `level` に設定する。 |
| `addHandler(hdlr)` | 指定されたハンドラ `hdlr` をこのロガーに追加する。このメソッドを何回も呼び出して複数の異なるハンドラを追加することもできる。<br />ロガーは、全てのハンドラに同じログ記録の出力処理を委ねる |
| `removeHandler(hdlr)` | 指定されたハンドラ `hdlr` をこのロガーから取り除く |
| `addFilter(filter)` | 指定されたフィルター `filter` をこのロガーに追加する。このメソッドを何回も呼び出して複数の異なるフィルターを追加することも<br />できる。ロガーは、全てのフィルターを通過したログ記録のみハンドラに処理を委ねる |
| `removeFilter(filter)` | 指定されたフィルター `filter` をこのロガーから除去する |

実のところ、有効レベルの概念とイベントの伝播の仕組みがあるため、上記のメソッドを全てのロガーで毎回呼び出す必要はない。

ロガーには、**有効レベル**（effective level）の概念がある。ロガーにレベルが明示的に設定されていなければ、代わりに親のレベルがその有効レベルとして使われる。親のレベルが設定されなければ、その親のレベルが確かめられ、以下同様に、明示的に設定されたレベルが見つかるまで祖先が探索される。ルートロガーは、必ず明示的なレベルが設定されている（デフォルトでは `logging.WARNING`）。イベントを処理するかを決定するとき、ロガーの有効レベルを使って、イベントがロガーのハンドラに渡されるかが決められる。

たとえば、最低のログレベルを `logging.DEBUG` と設定したハンドラ `st_handler` をロガーに取り付け、ロガーで最低のログレベルを指定しない場合:

``` python
logger = logging.getLogger(__name__)
logger.addHandler(st_handler)
```

この場合、モジュールレベルロガー `logger` の有効レベルは、ルートロガーのデフォルトの `logging.WARNING` とされる結果、`logging.INFO` ログは出力されない。

ロガー階層では、ロガーのログレベルとフィルターの設定を満たした場合に限り、そのロガーに記録されたイベントが親ロガーのハンドラにも渡される。これを**イベントの伝播**（event propagation）と呼ぶ。イベントの伝播は、親ロガーを経由するのではなく、親ロガーのハンドラに直接メッセージを渡して出力処理を委ねるものなので、**親ロガーのレベルとフィルターは作用しない**。

イベントは親ロガーに渡された後、さらにその親にも渡される。ハンドラをロガーとその祖先ロガーに取り付けた場合、同一レコードが複数回出力される場合がある。一般的に、ハンドラを複数のロガーに取り付ける必要はない。ハンドラがないことでイベントの伝播は止まらない。しかし、この連鎖構造において、祖先ロガーの `propagate` 属性が `False` に設定されていた場合、そのロガーのハンドラがイベントを処理する最後のハンドラとなり、その時点でイベントの伝播は止まる。

ロガーの `propagate` 属性の値はデフォルトで `True` に設定される。 `propagate` 属性の設定が `True` のままになっていれば、ロガー階層において最上位にある適切なロガーにハンドラを取り付けるだけで、そのハンドラは全ての子孫ロガーが記録する全てのイベントを確認することができる。一般的なシナリオでは、ハンドラをルートロガーに対してのみ取り付け、残りはイベントの伝播にすべて委ねる。こうすることで、アプリケーションが使用する全てのライブラリとアプリケーション自身のログ記録が統一的な基準で運用される。ロガーの `propagate` 属性を `False` に設定することで、伝播を抑制することもできる。

ルートロガーの設定は、上記のメソッド群を呼び出すのではなく、次のモジュール関数を 1 回呼び出すだけで簡単に行うことができる。

``` python
logging.basicConfig(**kwargs)
```

この関数は、ハンドラとフォーマッターを作成し、そのハンドラのフォーマッターを設定した上で、ルートロガーにそのハンドラを取り付ける。

以下のキーワード引数がサポートされる。

| 引数 | 意味 |
|:---|:---|
| `filename` | 指定されたファイルパスで `FileHandler` オブジェクトが作成される。デフォルトでは `StreamHandler` が作成される |
| `filemode` | `filename` が指定された場合、このモードでファイルが開かれる。デフォルトは `'a'` |
| `encoding` | `filename` が指定された場合、このエンコーディングが使用される（Python 3.9 で追加） |
| `errors` | `filename` が指定された場合、このエラーモードが使用される（Python 3.9 で追加） |
| `stream` | 指定されたストリームを `StreamHandler` の初期化に使う。この引数は `filename` と同時には使えない。両方が指定されたときには `ValueError` が送<br />出される |
| `handlers` | ルートロガーに追加される既に作られたハンドラのイテラブル。まだフォーマッターがセットされていない全てのハンドラは、この関数で作られたデフォ<br />ルトフォーマッターが割り当てられることになる。この引数は `filename` や `stream` と互換性がない。両方が存在する場合 `ValueError` が送出される |
| `format` | 作成されるフォーマッターの書式文字列。デフォルトは `%(levelname)s:%(name)s:%(message)s` |
| `datefmt` | 作成されるフォーマッターの日時書式 |
| `style` | 作成されるフォーマッターのプレースホルダーの形式 |
| `level` | ルートロガーのレベルを `level` に設定する |
| `force` | `True` を設定する場合、ルートロガーに取り付けられたハンドラは全て取り除かれ、他の引数によって指定された設定が有効になる。デフォルトでは、こ<br />の関数はルートロガーに設定されたハンドラがあれば何もしない |

`logging` を使うライブラリ `foo` による全てのロギングが、 `'foo.x'`, `'foo.x.y'` その他に該当する名前のロガーによってなされるなら、次のように `NullHandler` だけを設定する。

``` python
import logging
logging.getLogger('foo').addHandler(logging.NullHandler())
```

### ログメッセージの生成 ###

ロガーが設定されれば、以下のメソッドがログメッセージを生成し、残りの処理がハンドラに委ねられる（つまり書式化と出力処理までが行われる）。

| メソッド | 機能 |
|:---|:---|
| `debug(msg, *args, **kwargs)`<br />`info(msg, *args, **kwargs)`<br />`warning(msg, *args, **kwargs)`<br />`error(msg, *args, **kwargs)`<br />`critical(msg, *args, **kwargs)` | メッセージ `msg` とメソッド名に対応したレベルでログ記録を作り出す。`msg` は `'%s'`, `'%d'`, `'%f'` などの % 変換指定を<br />含むことができ、それらが登場する順に、後続の位置引数に、置き換える値で評価されるオブジェクトを指定する |
| `log(level, msg, *args, **kwargs)` | `level` レベルでログ記録を作り出す。その他の引数は `debug()` と同じように解釈される |
| `exception(msg, *args, **kwargs)` | `error()` と同様に `logging.ERROR` レベルでログ記録を作り出すが、スタックトレースを一緒にダンプする。このメソッド<br />は例外ハンドラからだけ呼び出すようにすること |

In [ ]:
# ロガー
logger = logging.getLogger("sample")

# ルートロガーの設定
# st_handler のレベル設定 DEBUG よりこのレベル設定 INFO が優先される
# Colab でルートロガーが設定されているので、force 引数で True を指定する
logging.basicConfig(level=logging.INFO, handlers=[st_handler], force=True)

# ログ記録の作成
items = [("John Does", 198862), ("Jane Smith", 793901)]
logger.info("Number of Items: %s", len(items))
logger.info("%s password = %d", items[0][0], items[0][1])  # フィルターにより出力されない
logger.debug("some debug message")  # INFO レベルより低いメッセージは出力されない

2024-08-14 02:23:10,442 - sample - INFO - Number of Items: 2


メッセージで % 変換指定を使用する代わりに f-string を使用することはやってはならない。たとえば

``` python
logger.debug(f"Number of Items: {len(items)}")
```

のように f-string を指定した場合、このメソッドを呼び出す際に引数が評価され、書式の評価が行われる。この評価は、実際にログが出力されるべきかどうかに関わらず行われるから、ログが出力されるべきでないときは無駄になる。`str.format()` で書式化された文字列を使用する場合も同様である。

メッセージに % 変換指定を含め、位置引数を渡す場合、`debug()` などは、実際にログが出力されるまで、`msg` の書式を評価しないので、書式の評価が無駄になることがない。なお、後続の位置引数が提供されない場合はメッセージの % 変換は実行されない。

`kwargs` には、以下のキーワード引数を指定できる。

| キーワード引数 | 意味 |
|:---|:---|
| `exc_info` | `True` を指定した場合、`exception()` と同様に例外発生時のスタックトレースをダンプする |
| `extra` | イベントで生成される `LogRecord` の `__dict__` にユーザー定義属性を加えるのに使われる辞書を指定できる。フォーマッターは `LogRecord` の<br />ユーザー定義属性も処理する |

次のコードは、 `exc_info` キーワード引数の使用例である:

In [ ]:
import logging

logger = logging.getLogger("sample")

def foo():
    bar()

def bar():
    logger.info("bar")
    1 / 0

if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, force=True)
    try:
        foo()
    except:
        logger.error("hmm", exc_info=True)
        # raise  # 実際には例外を再送出すべき

INFO:sample:bar
ERROR:sample:hmm
Traceback (most recent call last):
  File "<ipython-input-4-8854c162baac>", line 15, in <cell line: 12>
    foo()
  File "<ipython-input-4-8854c162baac>", line 6, in foo
    bar()
  File "<ipython-input-4-8854c162baac>", line 10, in bar
    1 / 0
ZeroDivisionError: division by zero


これは `exception()` メソッドで書き換えることができる:

In [ ]:
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO, force=True)
    try:
        foo()
    except:
        logger.exception("hmm", exc_info=True)
        # raise  # 実際には例外を再送出すべき

INFO:sample:bar
ERROR:sample:hmm
Traceback (most recent call last):
  File "<ipython-input-5-18ba18d492fb>", line 4, in <cell line: 1>
    foo()
  File "<ipython-input-4-8854c162baac>", line 6, in foo
    bar()
  File "<ipython-input-4-8854c162baac>", line 10, in bar
    1 / 0
ZeroDivisionError: division by zero


次のコードは、 `extra` キーワード引数の使用例である:

In [ ]:
import logging
FORMAT = '%(asctime)s %(clientip)-15s %(user)-8s %(message)s'
d = {'clientip': '192.168.0.1', 'user': 'fbloggs'}
logger = logging.getLogger('tcpserver')
logging.basicConfig(format=FORMAT, force=True)
logger.warning('Protocol problem: %s', 'connection reset', extra=d)

2024-08-14 02:23:10,508 192.168.0.1     fbloggs  Protocol problem: connection reset


``` python
logging.debug(msg, *args, **kwargs)
logging.info(msg, *args, **kwargs)
logging.warning(msg, *args, **kwargs)
logging.error(msg, *args, **kwargs)
logging.critical(msg, *args, **kwargs)
logging.exception(msg, *args, **kwargs)
logging.log(level, msg, *args, **kwargs)
```

これらのモジュール関数は、それぞれルートロガーの同名のメソッドを呼び出す。ルートロガーにハンドラが設定されていない場合に自動的に `basicConfig()` を呼び出す。ライブラリでこれらの関数を使用すべきではない。

公式の Logging クックブックの次の章には、 `DEBUG` よりも高いレベルのメッセージはファイルに記録し、`INFO` 以上のレベルのメッセージはコンソールに出力したいという場合（また、ファイルにはタイムスタンプを記録し、コンソールには出力しないとする）のロガー設定レシピが掲載されている。

  * [複数の出力先にログを出力する](https://docs.python.org/ja/3/howto/logging-cookbook.html#logging-to-multiple-destinations)

### ロギングの環境設定 ###

``` python
logging.config.dictConfig(config)
```

このサブモジュール `logging.config` が提供する関数は、辞書 `config` からロギング環境設定を取得する。

`config` は以下のキーを含む辞書である。`'version'` キーだけ必須。

| キー | 値 |
|:---|:---|
| `'version'` | （必須）環境設定辞書の形式のバージョン。現在 `1` のみサポートされる |
| `'formatters'` | 辞書であり、キーは個々のフォーマッターの名前、その値はフォーマッターの設定を辞書で表したもの。値に使う辞書のキーは、<br />`logging.Formatter()` の引数の名前が使える（ただし、`fmt` は `'format'` とする） |
| `'filters'` | 辞書であり、キーは個々のフィルターの名前、その値はフィルターの設定を辞書で表したもの。値に使う辞書のキーは、<br />`logging.Filter()` の引数の名前が使える |
| `'handlers'` | 辞書であり、キーは個々のハンドラの名前、その値はハンドラの設定を辞書で表したもの。値に使う辞書のキーは次のとおり<br /><br />・`'class'`（必須）: `'logging.'` から始めるハンドラクラスの名前<br /><br />・`'level'`（任意）: ハンドラのレベル<br /><br />・`'formatter'`（任意）: ハンドラに追加するフォーマッターの名前<br /><br />・`'filters'`（任意）: ハンドラに追加するフィルターの名前のリスト<br /><br />・その他のすべてのキー（任意）: ハンドラのコンストラクタにキーワード引数として渡される |
| `'loggers'` | 辞書であり、キーは個々のロガーの名前、その値はロガーの設定を辞書で表したもの。値に使う辞書のキーは次のとおり<br /><br />・`'level'`（任意）: ロガーのレベル<br /><br />・`'propagate'`（任意）: ロガーの `propagate` 属性の設定。0 なら伝播させない<br /><br />・`'filters'`（任意）: ロガーに追加するフィルターの名前のリスト<br /><br />・`'handlers'`（任意）: ロガーに追加するハンドラの名前のリスト |
| `'root'` | ルートロガーへの設定。`'propagate'` キーによる設定が適用されないことを除き、`'loggers'` と同じ |
| `'incremental'` | `True` の場合、この環境設定が既存のロギング設定に対する増分として解釈される。具体的には、`formatters` と `filters` の<br />項目は無視され、`handlers` の項目の `level` 設定と、`loggers` と `root` の項目の `level` と `propagate` 設定のみが処理され<br />る。`False`（デフォルト）の場合、既存のロギング設定を置き換える |
| `'disable_existing_loggers'` | `True`（デフォルト）の場合、ルートロガー以外の既存のロガーは全て無効化され、この環境設定で設定したロガーだけが有効で<br />ある。`False` の場合、既存のロガーは有効のまま残される。この値は、`incremental` が `True` なら無視される |

JSON 形式 や TOML 形式などで設定ファイルを書き、そのファイルを開いて内容を変換した辞書を `logging.config.dictConfig()` に渡すようにすれば、ロギング環境設定とスクリプトのコードを分離することができ、ロギングの設定を変えやすくなる。

たとえば、以下のコードでは、`'hogeLogger'` の名前でロガーの設定が書いてある JSON 形式のファイルから設定を読み込んでいる。

``` python
import logging.config
import json

with open("logging_config.json") as f:
    logging.config.dictConfig(json.load(f))

logger = getLogger("hogeLogger")
```

設定ファイル（.json）の例:

``` json
{
  "version":1,
  "disable_existing_loggers": false,
  "formatters": {
    "simple": {
      "format": "%(asctime)s %(name)s:%(lineno)s %(funcName)s [%(levelname)s]: %(message)s"
    }
  },
  "filters": {
    "layer-filter": {
      "name": "hogeLogger"
    }
  },
  "handlers": {
    "consoleHandler": {
      "class": "logging.StreamHandler",
      "level": "DEBUG",
      "formatter": "simple",
      "stream": "ext://sys.stdout"
    },

    "fileHandler": {
      "class": "logging.FileHandler",
      "level": "DEBUG",
      "formatter": "simple",
      "filters": ["layer-filter"],
      "filename": "app.log",
      "encoding": "utf-8"
    }
  },
  "loggers": {
    "hogeLogger": {
      "level": "DEBUG",
      "handlers": ["consoleHandler", "fileHandler"]
    },
    "fooLogger": {
      "level": "DEBUG",
      "handlers": ["consoleHandler", "fileHandler"]
    }
  },
  "root": {
    "level": "ERROR"
  }
}
```

### スレッドセーフ性 ###

ハンドラはスレッドセーフで、複数スレッドのロギングが互いを邪魔することがない。このスレッドセーフ性は、`threading` の再入可能ロックによって達成されている。モジュールの共有データへのアクセスを直列化するためのロックが 1 つ存在し、各ハンドラでも背後にある I/O へのアクセスを直列化するためにロックを生成している。このため、各スレッドからのメッセージが区別されて安全に出力される。

In [ ]:
import logging
import time
import threading

# ログの設定
logger = logging.getLogger("multithreaded")

def worker1():
    logger.info("start")
    time.sleep(0.2)
    logger.info("end")

def worker2():
    logger.info("START")
    time.sleep(0.2)
    logger.info("END")

def main():
    FORMAT = "[%(levelname)s] %(threadName)s: %(message)s"
    logging.basicConfig(level=logging.INFO, format=FORMAT, force=True)
    # マルチスレッド
    t1 = threading.Thread(target=worker1, name="t1")
    t2 = threading.Thread(target=worker2, name="t2")
    t1.start()
    t2.start()
    t1.join()
    t2.join()

if __name__ == "__main__":
    main()

[INFO] t1: start
[INFO] t2: START
[INFO] t1: end
[INFO] t2: END


追加のハンドラ
--------------

`logging` のサブモジュール `logging.handlers` は、以下の `logging.Handler` のサブクラスを追加している。

| ハンドラクラス | 機能 |
|:---|:---|
| `RotatingFileHandler` | メッセージをディスクファイルに送り、最大ログファイル数とログファイル循環をサポートする |
| `TimedRotatingFileHandler` | メッセージをディスクファイルに送り、ログファイルを特定時間のインターバルで循環する |
| `SocketHandler` | TCP/IP ソケットにメッセージを送る。Unix ドメインソケットもサポートされる |
| `DatagramHandler` | UDP ソケットにメッセージを送る。Unix ドメインソケットもサポートされる |
| `SMTPHandler` | メッセージを指示された email アドレスに送る |
| `SysLogHandler` | メッセージを（必要ならばリモートマシンの） Unix syslog daemon に送る |
| `NTEventLogHandler` | メッセージを Windows NT/2000/XP イベントログに送る |
| `MemoryHandler` | メッセージを、特定の基準が満たされる度に流される、メモリ中のバッファに送る |
| `HTTPHandler` | メッセージを、 GET または POST セマンティクスを使って、HTTP サーバに送る |
| `WatchedFileHandler` | ロギングする先のファイルを監視する。ファイルが変更されると、そのファイルは閉じられ、ファイル名を使って再び開かれる。<br />Unix 系 OS だけでサポートされる |
| `QueueHandler` | `queue` モジュールや `multiprocessing` モジュールなどで実装されているキューにメッセージを送る |

全容は[公式ドキュメント](https://docs.python.org/ja/3/library/logging.handlers.html)を参照。

### RotatingFileHandler ###

``` python
logging.handlers.RotatingFileHandler(filename, mode='a', maxBytes=0, backupCount=0, encoding=None, delay=False, errors=None)
```

ログ出力先をファイルシステム上のファイルとし、ログファイルが指定されたサイズに達すると、ロールオーバー（ファイルが閉じられ、暗黙のうちに新たなファイルが開かれて、新しいファイルにログ出力が切り替わること）が行われるようなハンドラを返す。`logging.FileHandler()` と同様の引数に加えて、以下の引数も指定できる。

| 引数 | 意味 |
|:---|:---|
| `maxBytes` | `maxBytes` でファイルをロールオーバーする。`maxBytes` が 0 （デフォルト）ならロールオーバーは起きない |
| `backupCount` | `backupCount` 個までファイルの履歴を残す。たとえば、`filename` が `app.log` で、`backupCount` が 3 なら、`app.log` が満杯になると閉じられ、<br />`app.log.1` に名前が変更されるが、`app.log.1`, `app.log.2` が存在する場合、それらのファイルはそれぞれ `app.log.2`, `app.log.3` といった<br />具合に名前が変更される。ログ出力先となるファイルは常に `app.log` である。 `backupCount` が 0 （デフォルト）ならロールオーバーは起きない |

### TimedRotatingFileHandler ###

``` python
logging.handlers.TimedRotatingFileHandler(filename, when='h', interval=1, backupCount=0, encoding=None, delay=False, utc=False, atTime=None, errors=None)
```

ログ出力先をファイルシステム上のファイルとし、指定された時間や間隔に基づいてログファイルをロールオーバーするようなハンドラを返す。古いログファイルの保存時、ファイル名に日付と時間に基づいた拡張子が付けられる。`logging.FileHandler()` と同様の引数に加えて、以下の引数も指定できる。

| 引数 | 意味 |
|:---|:---|
| `interval` | ロールオーバーする時間または間隔 |
| `when` | `interval` の単位を指定する。使える値は以下の通りで、大小文字の区別は行われない<br /><br />・`'S'`: 秒<br /><br />・`'M'`: 分<br /><br />・`'H'`: 時間<br /><br />・`'D'`: 日<br /><br />・`'W0'-'W6'`: 曜日（0=月曜） ※これを指定する場合 `interval` は使われない<br /><br />・`'midnight'`: `atTime` が指定されなかった場合は深夜に、そうでない場合は `atTime` の時刻にロールオーバーされる） ※これを指定する場合<br />　 `interval` は使われない |
| `backupCount` | 保存されるファイル数。それ以上のファイルがロールオーバーされる時に作られるならば、一番古いものが削除される。0 なら古いファイルが保<br />存されない |
| `utc` | `True` の場合、時刻は UTC になる。`False`（デフォルト）の場合、現地時間が使われる |
| `atTime` | `when` が 'W0'-'W6' または `'midnight'` の場合、`atTime` が初期のロールオーバー時刻の算出に使用される。`atTime` を指定するなら、それは<br /> `datetime.time` インスタンスでなければならない |

### SMTPHandler ###

``` python
logging.handlers.SMTPHandler(mailhost, fromaddr, toaddrs, subject, credentials=None, secure=None, timeout=1.0)
```

ログ出力先を特定のメールアドレスとするハンドラを返す。

| 引数 | 意味 |
|:---|:---|
| `mailhost` | SMTP サーバーを文字列で指定する。非標準の SMTP ポートを指定するには、`(host, port)` のタプル形式を指定する |
| `fromaddr` | 送信元メールアドレスを文字列で指定する |
| `toaddrs` | ログを送信する先のメールアドレスのリスト（文字列からなるリスト）を指定する |
| `subject` | メールのタイトルを文字列で指定する |
| `credentials` | SMTP サーバーが認証を必要とする場合、`(username, password)` のタプル形式を指定できる |
| `secure` | セキュアプロトコル（TLS）の使用をタプルで指定する |
| `timeout` | タイムアウトを浮動小数点数で指定する。デフォルトでは 1.0 |

### QueueHandler ###

``` python
logging.handlers.QueueHandler(queue)
```

ログ出力先をキューとするハンドラを返す。`queue` 引数には `queue` モジュールのキューまたは `multiprocessing` パッケージのキューを指定できる。キューへの書き込みは通常すぐに完了する（内部で `put_nowait()` を呼び出すので `queue.Full` 例外を捕捉する必要があるかもしれない）。このため、 `RotatingFileHandler` や `SMTPHandler` など動作が重いハンドラの代わりに使用される。

キューからログメッセージを受信する機能は、**リスナー**と呼ばれ次のクラスのインスタンスがサポートする。

``` python
logging.handlers.QueueListener(queue, *handlers, respect_handler_level=False)
```

| 引数 | 意味 |
|:---|:---|
| `queue` | `QueueHandler` のインスタンス化の際に渡したキューを指定する |
| `handlers` | キューに格納されたエントリーを処理するハンドラ。複数指定可 |
| `respect_handler_level` | `True` の場合、メッセージをハンドラーに渡すか決める時に（メッセージのレベルに比べて）ハンドラーのレベルが尊重される。<br />`False`（デフォルト）の場合、全てのメッセージは常にハンドラーに渡される |

`QueueListener` は、別スレッドを起動し、そのスレッドでキューから `LogRecord` を取り出し `handler` に渡して処理させる。 `QueueListener` インスタンスの主なメソッドは次のとおり。

| メソッド | 機能 |
|:---|:---|
| `start()` | リスナーを開始する。これはキューを監視するためにバックグラウンドスレッドを開始する |
| `stop()` | リスナーを停止する。スレッドに終了するように依頼し、終了するまで待つ。アプリケーションの終了前に必ずこのメソッドを呼ぶこと |

次のコードは、この 2 つのクラスを利用する例である。

In [ ]:
import logging
from logging.handlers import QueueHandler, QueueListener
import queue
que = queue.Queue()
queue_handler = QueueHandler(que)
handler = logging.StreamHandler()
listener = QueueListener(que, handler)
formatter = logging.Formatter("%(threadName)s: %(message)s")
queue_handler.setFormatter(formatter)
logger = logging.getLogger("queue_test")
logging.basicConfig(handlers=[queue_handler], force=True)
listener.start()
logger.warning("Look out!")
listener.stop()

MainThread: Look out!


ログ記録はスレッドセーフであり、単一プロセスの複数のスレッドから単一ファイルへのログ記録はサポートされているが、**複数プロセスから単一ファイルへのログ記録はサポートされない**。なぜなら、複数のプロセスをまたいで単一のファイルへのアクセスを直列化する標準の方法が Python には存在しないからである。

複数のプロセスから単一ファイルへログ記録しなければならないなら、`multiprocessing.Queue` と `QueueHandler` を使って、マルチプロセスアプリケーションの 1 つのプロセスに全ての `logging` イベントを送るようにする。

次の例はこれを行う方法を示す。この例ではメインプロセスが他のプロセスから送られたイベントを受け取り、それを独自のロギング設定にしたがって保存する。

In [ ]:
# fmt: on
# You'll need these imports in your own code
import logging
import logging.handlers
import multiprocessing


# サブプロセス
def worker_process(queue):
    # プロセスでのルートロガーの設定--QueueHandlerを使用
    handler = logging.handlers.QueueHandler(queue)
    logging.basicConfig(level=logging.DEBUG, format="", handlers=[handler], force=True)
    # ログメッセージの生成
    logger = logging.getLogger("multiprocess_test")
    logger.info("message from %s", logger.name)


def main():
    # リスナーの設定
    handler = logging.handlers.RotatingFileHandler("mptest.log", "a", 3000, 10)
    formatter = logging.Formatter("%(asctime)s %(processName)-10s %(levelname)-8s %(message)s")
    handler.setFormatter(formatter)
    queue = multiprocessing.Queue()
    listener = logging.handlers.QueueListener(queue, handler)
    # リスナーの開始
    listener.start()

    workers = []
    for i in range(10):
        worker = multiprocessing.Process(target=worker_process, name=f"Process{i}", args=(queue,))
        workers.append(worker)
        worker.start()
    for w in workers:
        w.join()

    # リスナーの停止
    listener.stop()

    # おまけ: ログファイルを読み込んでファイルが壊れていないことを確認する
    with open("mptest.log") as f:
        for line in f.readlines():
            print(line.strip())


if __name__ == "__main__":
    main()
!rm mptest.log

2024-08-14 09:57:02,002 Process0   INFO     message from multiprocess_test
2024-08-14 09:57:02,016 Process1   INFO     message from multiprocess_test
2024-08-14 09:57:02,018 Process2   INFO     message from multiprocess_test
2024-08-14 09:57:02,024 Process3   INFO     message from multiprocess_test
2024-08-14 09:57:02,031 Process4   INFO     message from multiprocess_test
2024-08-14 09:57:02,046 Process6   INFO     message from multiprocess_test
2024-08-14 09:57:02,052 Process7   INFO     message from multiprocess_test
2024-08-14 09:57:02,043 Process5   INFO     message from multiprocess_test
2024-08-14 09:57:02,058 Process8   INFO     message from multiprocess_test
2024-08-14 09:57:02,064 Process9   INFO     message from multiprocess_test


asyncio のログ記録
------------------

`asyncio` は `logging` モジュールを利用し、全てのログ記録は `"asyncio"` ロガーを通じて行われる。デフォルトのログレベルは `logging.INFO` であるが、これは簡単に調節できる:

``` python
logging.getLogger("asyncio").setLevel(logging.WARNING)
```

ネットワークログ記録は、イベントループをブロックすることがある。 `QueueHandler` と `QueueListener` を使用することが推奨される。

`asyncio` のデバッグモードを有効化すると、`logging.DEBUG` レベルで以下のようなログが出力される。

  * `await` を伴わないコルーチンがないかをチェックし、それらが記録される。
  * I/O 多重化実装が I/O 処理を実行する時間が長すぎる場合、その実行時間が記録される。
  * 実行時間が 100 ミリ秒を超えるコールバックが記録される。この秒数はイベントループの属性 `loop.slow_callback_duration` で設定できる。

`asyncio` ロガーのログレベルが `logging.DEBUG` に設定されるようにすること。たとえば、アプリケーションの起動時に以下を実行する:

``` python
logging.basicConfig(level=logging.DEBUG)
```

`asyncio` のデバッグモードを有効化する方法はいくつかある:

  * `PYTHONASYNCIODEBUG` 環境変数の値を `1` に設定する。
  * `Python` 開発モード を使う。コマンドラインで `-X dev` フラグを指定して Python を起動する。
  * `asyncio.run()` 実行時に `debug=True` を設定する。
  * イベントループのメソッド `loop.set_debug()` を呼び出す。

スタックトレースの抽出
----------------------

`logger.exception()` で例外の内容をログに書き出すことができるが、連鎖した例外のトレースバックが全てダンプされる。

In [ ]:
import logging
import traceback

logger = logging.getLogger("traceback1")
logging.basicConfig(level=logging.ERROR, force=True)

try:
    raise TypeError
except Exception:
    try:
        raise ValueError
    except Exception:
        logger.exception("exceptionで出力")  # 連鎖した例外のトレースバックを全てダンプする

ERROR:traceback1:exceptionで出力
Traceback (most recent call last):
  File "<ipython-input-8-e53b192554b3>", line 8, in <cell line: 7>
    raise TypeError
TypeError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<ipython-input-8-e53b192554b3>", line 11, in <cell line: 7>
    raise ValueError
ValueError


`logger.exception()` より柔軟にスタックトレースを扱うには、標準ライブラリの `traceback` モジュールを使用する。


``` python
traceback.format_exc(limit=None, chain=True)
```

この関数は、発生した例外からスタックトレースを抽出し、書式を整えた文字列を返す。

| 引数 | 意味 |
|:---|:---|
| `limit` | 指定した数までのスタックトレースを抽出する。`None`（デフォルト）の場合はすべてのスタックトレースを抽出する |
| `chain` | `True`（デフォルト）の場合、連鎖した例外も出力する |

例外の連鎖において、except 句で直接捕捉した例外のトレースバックだけをダンプしたい場合は、`format_exc()` 関数を `chain=False` と指定して呼び出す。

In [ ]:
import logging
import traceback

logger = logging.getLogger("traceback2")
logging.basicConfig(level=logging.ERROR, force=True)

try:
    raise TypeError
except Exception:
    try:
        raise ValueError
    except Exception:
        tb = traceback.format_exc(chain=False)
        logger.error("format_excを使用\n%s", tb)

ERROR:traceback2:format_excを使用
Traceback (most recent call last):
  File "<ipython-input-9-9a6589859eac>", line 11, in <cell line: 7>
    raise ValueError
ValueError





``` python
traceback.print_exc(limit=None, file=None, chain=True)
```

この関数は、発生した例外からスタックトレースを抽出し、`file` に出力する。`file` は書き込み可能で開いたファイルオブジェクトでなければならない。`file` が `None`（デフォルト）の場合、 `sys.stderr` が使われる。

`traceback` モジュールは例外をキャプチャするクラス `traceback.TracebackException` も提供している。このクラスは、次のクラスメソッドを使うと簡単にインスタンスを生成することができる。

``` python
traceback.TracebackException.from_exception(exc, *, limit=None, lookup_lines=True, capture_locals=False)
```

| 引数 | 意味 |
|:---|:---|
| `exc` | 例外オブジェクトを指定する |
| `limit` | 指定した数までのスタックトレースをキャプチャする。`None`（デフォルト）の場合、全てのスタックトレースをキャプチャする |
| `lookup_lines` | `True`（デフォルト）の場合、全てのスタックトレースを読み込む。`False` の場合、レンダリングされるまで読み込まれない |
| `capture_locals` | `True` の場合、各スタックトレースのローカル変数がオブジェクト表現としてキャプチャされる |

`traceback.TracebackException` の主なメソッドは次のとおり。

| メソッド | 機能 | 戻り値 |
|:---|:---|:---|
| `format(*, chain=True)` | 例外を書式化する | 文字列のジェネレーター |
| `print(*, file=None, chain=True)` | 例外の書式化を出力する | `None` |

次のコードは `TracebackException` の使用例である。

In [ ]:
import logging
import traceback
import sys

logger = logging.getLogger("TracebackException")
logging.basicConfig(level=logging.INFO, force=True)

try:
    raise TypeError
except Exception:
    try:
        raise ValueError
    except Exception:
        exc_info = sys.exc_info()
        tb = "".join(traceback.TracebackException.from_exception(exc_info[1]).format(chain=False))
        logger.error("TracebackExceptionを使用\n%s", tb)

ERROR:TracebackException:TracebackExceptionを使用
Traceback (most recent call last):
  File "<ipython-input-10-a48c8d50b06e>", line 12, in <cell line: 8>
    raise ValueError
ValueError

